
# Frage 1 – Notebook 01: Daten einlesen & vorbereiten

Dieses Notebook liest die **Rohdaten** (Excel) ein, bereinigt die Jahresangaben, baut einen **Preisindex (CPI)** aus den Inflationsraten und erzeugt eine zusammengeführte, inflationsbereinigte Tabelle als CSV (`warenkorb_auswertung.csv`).  
Die CSV ist die gemeinsame Grundlage für die weiteren Notebooks in *frage1* (Aufbereitung/Normalisierung, Plots, Tests).



## Voraussetzungen

- Die folgenden Dateien liegen im gleichen Ordner wie dieses Notebook:
  - `Inflationsrate.xlsx`
  - `Umsatz Lebensmitteleinzelhandel.xlsx`
  - `Umsatzentwicklung im Einzelhandel.xlsx`
  - `Konsumausgaben Bekleidung und Schuhe.xlsx`
  - `Konsumausgaben Nahrungsmittel, Getränke Tabakwaren Drogen.xlsx`


In [19]:

import pandas as pd
import numpy as np

# Anzeige-Einstellungen
pd.set_option("display.max_rows", 8)
pd.set_option("display.width", 120)

from pathlib import Path
import pandas as pd

# Projekt-Root ermitteln (anpassen falls nötig)
project_root = Path("D:/JupyterProject")

# Datenordner
data_path = project_root / "data"


## 1) Daten laden

In [20]:

inflation = pd.read_excel(
    data_path / "Inflationsrate.xlsx", sheet_name=1, skiprows=4, usecols="B:C", names=["Jahr", "Inflation"]
)

lm = pd.read_excel(
    data_path / "Umsatz Lebensmitteleinzelhandel.xlsx", sheet_name=1, skiprows=4, usecols="B:C",
    names=["Jahr", "LM_Umsatz_nom"]
)

einzel_vj = pd.read_excel(
    data_path / "Umsatzentwicklung im Einzelhandel.xlsx", sheet_name=1, skiprows=4, usecols="B:C",
    names=["Jahr", "EH_Veraenderung_%"]
)

bekl = pd.read_excel(
    data_path / "Konsumausgaben Bekleidung und Schuhe.xlsx", sheet_name=1, skiprows=4, usecols="B:C",
    names=["Jahr", "Bekl_nom"]
)

food = pd.read_excel(
    data_path / "Konsumausgaben Nahrungsmittel, Getränke Tabakwaren Drogen.xlsx",
    sheet_name=1, skiprows=4, usecols="B:C",
    names=["Jahr", "Food_nom"]
)

inflation.head(), lm.head(), bekl.head(), food.head()


(  Jahr  Inflation
 0  '92        5.0
 1  '93        4.5
 2  '94        2.7
 3  '95        1.9
 4  '96        1.4,
    Jahr  LM_Umsatz_nom
 0  2002      126565.22
 1  2003      136057.29
 2  2004      136303.04
 3  2005      145271.09
 4  2006      145685.06,
    Jahr  Bekl_nom
 0  1991     57.05
 1  1992     60.56
 2  1993     61.67
 3  1994     60.78
 4  1995     60.56,
    Jahr  Food_nom
 0  1991    144.74
 1  1995    153.63
 2  2000    167.71
 3  2001    174.24
 4  2002    175.91)

## 2) Bereinigung (Jahre & Numerik)

In [21]:

# a) Inflations-Jahre "'01" -> 2001
def fix_year(x):
    s = str(x).strip()
    if s.startswith("'") and len(s) == 3:  # z.B. '01
        return 2000 + int(s[1:])
    try:
        return int(s)
    except ValueError:
        return pd.NA

inflation["Jahr"] = inflation["Jahr"].apply(fix_year).astype("Int64")
inflation["Inflation"] = pd.to_numeric(inflation["Inflation"], errors="coerce")

# andere Tabellen: sicherstellen, dass Jahr int ist & Werte numerisch sind
for df in (lm, einzel_vj, bekl, food):
    df["Jahr"] = pd.to_numeric(df["Jahr"], errors="coerce").astype("Int64")
    for col in df.columns:
        if col != "Jahr":
            df[col] = pd.to_numeric(df[col], errors="coerce")

inflation.tail(3), lm.tail(3)


(    Jahr  Inflation
 30  2022        6.9
 31  2023        5.9
 32  2024        2.2,
     Jahr  LM_Umsatz_nom
 19  2021      233637.02
 20  2022      252202.65
 21  2023      271872.98)

## 3) Preisindex (CPI) aus Inflationsraten

In [22]:

# Gemeinsames Startjahr
first_year = max(lm["Jahr"].min(), bekl["Jahr"].min(), food["Jahr"].min(), inflation["Jahr"].min())

infl = inflation[inflation["Jahr"] >= first_year].sort_values("Jahr").reset_index(drop=True)
infl["CPI"] = (1 + infl["Inflation"] / 100.0).cumprod() * 100.0
base_cpi = infl["CPI"].iloc[0]
infl["CPI"] = infl["CPI"] / base_cpi * 100.0  # Basisjahr = 100

infl.head()


,Jahr,Inflation,CPI
0,2002,1.4,100.000000
1,2003,1.0,101.000000
2,2004,1.6,102.616000
3,2005,1.6,104.257856
4,2006,1.6,105.925982


## 4) Mergen & reale Werte rechnen

In [23]:

df = infl[["Jahr", "CPI"]].merge(lm, on="Jahr", how="inner")
df = df.merge(bekl, on="Jahr", how="inner")
df = df.merge(food, on="Jahr", how="inner")

# reale Werte = nominal * (100 / CPI)
df["LM_Umsatz_real"] = df["LM_Umsatz_nom"] * (100.0 / df["CPI"])
df["Bekl_real"]      = df["Bekl_nom"]      * (100.0 / df["CPI"])
df["Food_real"]      = df["Food_nom"]      * (100.0 / df["CPI"])

df.head()


,Jahr,CPI,LM_Umsatz_nom,Bekl_nom,Food_nom,LM_Umsatz_real,Bekl_real,Food_real
0,2002,100.000000,126565.22,62.51,175.91,126565.220000,62.510000,175.910000
1,2003,101.000000,136057.29,61.86,175.69,134710.188119,61.247525,173.950495
2,2004,102.616000,136303.04,60.59,178.54,132828.252904,59.045373,173.988462
3,2005,104.257856,145271.09,59.85,183.05,139338.267229,57.405746,175.574299
4,2006,105.925982,145685.06,61.00,185.74,137534.774441,57.587382,175.348859


## 5) Kurz-Auswertung (CAGR-Beispiel)

In [24]:

def cagr(series, years):
    series = series.dropna()
    years  = years.loc[series.index]
    s0, sN = float(series.iloc[0]), float(series.iloc[-1])
    n = int(years.iloc[-1] - years.iloc[0])
    if s0 <= 0 or n <= 0:
        return np.nan
    return (sN / s0) ** (1 / n) - 1

for label_nom, label_real in [
    ("LM_Umsatz_nom", "LM_Umsatz_real"),
    ("Bekl_nom", "Bekl_real"),
    ("Food_nom", "Food_real"),
]:
    g_nom  = cagr(df[label_nom], df["Jahr"])
    g_real = cagr(df[label_real], df["Jahr"])
    print(f"{label_nom}: CAGR nominal = {g_nom:.2%}, real = {g_real:.2%}")


LM_Umsatz_nom: CAGR nominal = 3.24%, real = 1.84%
Bekl_nom: CAGR nominal = 0.92%, real = -0.45%
Food_nom: CAGR nominal = 2.14%, real = 0.76%


## 6) Persistieren: saubere CSV für Folge-Notebooks

In [25]:

out_csv = "warenkorb_auswertung.csv"
df.to_csv(out_csv, index=False)
print("Gespeichert:", out_csv, "->", len(df), "Zeilen")
df.tail(3)


Gespeichert: warenkorb_auswertung.csv -> 19 Zeilen


,Jahr,CPI,LM_Umsatz_nom,Bekl_nom,Food_nom,LM_Umsatz_real,Bekl_real,Food_real
16,2018,125.489168,197272.60,79.97,232.57,157202.891515,63.726616,185.330738
17,2019,127.246016,216323.77,77.93,240.22,170004.356048,61.243568,188.783907
18,2020,127.882246,224559.32,73.65,257.71,175598.511118,57.592044,201.521328
